<a href="https://colab.research.google.com/github/kittimaxz/Study/blob/main/Test_Association_Rule_M01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mysql-connector-python
!pip install mlxtend
#!pip install apyori

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import itertools
#from apyori import apriori

In [3]:
# Establish the connection
conn = mysql.connector.connect(
    host='159.138.255.180',
    user='root',
    password='Vcb168899@',
    database='vcb_ba'
)

# Create a cursor object
cursor = conn.cursor()

# Define SQL query
query = "SELECT * FROM analysis_products"

# Execute the query and fetch data # ระยะเวลาในการดึงข้อมูลนั้นขึ้นอยู่กับขนาดของข้อมูล ยิ่งมีขนาดใหญ่ยิ่งใช้เวลานาน
cursor.execute(query)
rows = cursor.fetchall()

# Get column names
column_names = [desc[0] for desc in cursor.description]


# read sql
#df = pd.read_sql(query,conn)
file = pd.DataFrame(rows,columns=column_names)
# Close the cursor and connection
cursor.close()
conn.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
file.to_csv('data_file.csv', index=False) # change sql to csv # ไฟล์ csv จะถูกเก็บไว้ใน Folder ของไฟล์งานที่ทำอยู่
df = pd.read_csv('data_file.csv') # read csv file

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# แสดงข้อมูล
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,purchases_id,cars_id,baskets_id,purchase_create,transferred_date,chanal_purchase,th_code,level_num,gender,...,cat_sub1,cat_sub1_name,cat_sub2,cat_sub2_name,product_quantity,product_price,total_amount,country_num,country_name,ck
0,4129178,2248805,2337512,196678524095069,2021-12-27 10:13:01,2022-01-02 12:21:17,1688,TH403375,3,0,...,127372010.0,大码女装,1046799.0,大码连衣裙,1.0,43.06,43.06,1,THAILAND,0
1,4129179,2249918,2338921,196678524101901,2021-12-28 15:01:56,2022-01-02 12:21:18,1688,TH174119,4,0,...,1033273.0,布线产品,1033954.0,线卡、线扣,200.0,0.55,110.00,1,THAILAND,0
2,4129180,2249918,2338921,196678524101902,2021-12-28 15:01:56,2022-01-02 12:21:18,1688,TH174119,4,0,...,NaN,NaN,NaN,NaN,100.0,0.80,80.00,1,THAILAND,0
3,4129181,2249918,2338921,196678524101903,2021-12-28 15:01:56,2022-01-02 12:21:18,1688,TH174119,4,0,...,NaN,NaN,NaN,NaN,50.0,2.60,130.00,1,THAILAND,0
4,4129182,2249918,2338921,196678524101904,2021-12-28 15:01:56,2022-01-02 12:21:18,1688,TH174119,4,0,...,NaN,NaN,NaN,NaN,200.0,3.80,760.00,1,THAILAND,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020767,6149945,2816071,3081323,196678528089285,2024-08-07 23:00:02,2024-08-08 12:49:38,1688,TH313880,1,0,...,NaN,NaN,NaN,NaN,2500.0,0.43,1075.00,1,THAILAND,0
2020768,6149946,2816071,3081323,196678528089286,2024-08-07 23:00:02,2024-08-08 12:49:38,1688,TH313880,1,0,...,NaN,NaN,NaN,NaN,2500.0,0.53,1325.00,1,THAILAND,0
2020769,6149947,2816072,3081320,196678528123715,2024-08-07 23:00:10,2024-08-08 15:10:33,1688,TH313880,1,0,...,123924007.0,剪刀,1035507.0,其他剪刀,100.0,9.00,900.00,1,THAILAND,0
2020770,6149948,2816074,3083062,196678528135432,2024-08-08 00:34:28,2024-08-08 09:33:55,TAOBAO,TH398422,1,0,...,NaN,NaN,NaN,NaN,1.0,44.98,59.00,1,THAILAND,0


In [6]:
df.shape # จำนวนข้อมูลทั้งหมด

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2020772, 30)

In [7]:
# ตัดคอลัมน์ที่ไม่ได้ทิ้ง
data = df[['id','purchases_id','cars_id','baskets_id','transferred_date','th_code','chanal_purchase','po','so','chanal_bill','product_link','product_name','cat_main','cat_main_name','cat_sub1','cat_sub1_name','cat_sub2','cat_sub2_name','product_quantity','product_price','total_amount']]
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,purchases_id,cars_id,baskets_id,transferred_date,th_code,chanal_purchase,po,so,chanal_bill,...,product_name,cat_main,cat_main_name,cat_sub1,cat_sub1_name,cat_sub2,cat_sub2_name,product_quantity,product_price,total_amount
0,4129178,2248805,2337512,196678524095069,2022-01-02 12:21:17,TH403375,1688,21122700096,XLY211226100380,1,...,大码连衣裙,10166.0,女装,127372010.0,大码女装,1046799.0,大码连衣裙,1.0,43.06,43.06
1,4129179,2249918,2338921,196678524101901,2022-01-02 12:21:18,TH174119,1688,21122800487,XHY211228100359,1,...,线卡、线扣,57.0,电子元器件,1033273.0,布线产品,1033954.0,线卡、线扣,200.0,0.55,110.00
2,4129180,2249918,2338921,196678524101902,2022-01-02 12:21:18,TH174119,1688,21122800487,XHY211228100359,1,...,其他插座,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.80,80.00
3,4129181,2249918,2338921,196678524101903,2022-01-02 12:21:18,TH174119,1688,21122800487,XHY211228100359,1,...,其他插座,NaN,NaN,NaN,NaN,NaN,NaN,50.0,2.60,130.00
4,4129182,2249918,2338921,196678524101904,2022-01-02 12:21:18,TH174119,1688,21122800487,XHY211228100359,1,...,船型开关,NaN,NaN,NaN,NaN,NaN,NaN,200.0,3.80,760.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020767,6149945,2816071,3081323,196678528089285,2024-08-08 12:49:38,TH313880,1688,24080700634,XHY240805100117,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2500.0,0.43,1075.00
2020768,6149946,2816071,3081323,196678528089286,2024-08-08 12:49:38,TH313880,1688,24080700634,XHY240805100117,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2500.0,0.53,1325.00
2020769,6149947,2816072,3081320,196678528123715,2024-08-08 15:10:33,TH313880,1688,24080700635,XHY240805100114,1,...,NaN,59.0,五金、工具,123924007.0,剪刀,1035507.0,其他剪刀,100.0,9.00,900.00
2020770,6149948,2816074,3083062,196678528135432,2024-08-08 09:33:55,TH398422,TAOBAO,24080800001,XLY240806100836,1,...,kumikumiหวานลบอายุกดพับขัดแตะบินบินแขนเสื้อเสื...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,44.98,59.00


In [8]:
data.isnull().any() # True = NaN

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
id,False
purchases_id,False
cars_id,False
baskets_id,False
transferred_date,False
th_code,False
chanal_purchase,True
po,False
so,True
chanal_bill,False


In [9]:
# ลบค่าว่างทิ้งทั้งหมด
data_drop = data.dropna() # ใช้คำสั่ง drop แล้วยังมีค่าว่างอยู่
data_drop

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,purchases_id,cars_id,baskets_id,transferred_date,th_code,chanal_purchase,po,so,chanal_bill,...,product_name,cat_main,cat_main_name,cat_sub1,cat_sub1_name,cat_sub2,cat_sub2_name,product_quantity,product_price,total_amount
0,4129178,2248805,2337512,196678524095069,2022-01-02 12:21:17,TH403375,1688,21122700096,XLY211226100380,1,...,大码连衣裙,10166.0,女装,127372010.0,大码女装,1046799.0,大码连衣裙,1.0,43.06,43.06
1,4129179,2249918,2338921,196678524101901,2022-01-02 12:21:18,TH174119,1688,21122800487,XHY211228100359,1,...,线卡、线扣,57.0,电子元器件,1033273.0,布线产品,1033954.0,线卡、线扣,200.0,0.55,110.00
6,4129184,2250644,2340008,196678524106956,2022-01-02 12:19:20,TH100168,1688,21122900508,XLY211229100501,1,...,餐椅,13.0,家装、建材,123610020.0,餐厅家具,121486003.0,餐椅,12.0,53.44,641.28
7,4129185,2250661,2339780,196678524105638,2022-01-02 12:19:21,TH059370,1688,21122900525,XLY211229100273,1,...,冰袋、冰包、野餐包,15.0,日用百货,10287.0,餐具,1031741.0,冰袋、冰包、野餐包,2.0,50.00,100.00
8,4129186,2250661,2339780,196678524105639,2022-01-02 12:19:21,TH059370,1688,21122900525,XLY211229100273,1,...,冰袋、冰包、野餐包,15.0,日用百货,10287.0,餐具,1031741.0,冰袋、冰包、野餐包,1.0,50.00,50.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014392,6143570,2814392,3080565,196678527470152,2024-08-06 15:26:20,TH120467,1688,24080500274,XLY240803100494,1,...,摩托车安全用品,71.0,汽摩及配件,3304.0,摩托车配附件,1036933.0,摩托车安全用品,4.0,50.00,200.00
2014393,6143571,2814392,3080565,196678527470153,2024-08-06 15:26:20,TH120467,1688,24080500274,XLY240803100494,1,...,摩托车安全用品,71.0,汽摩及配件,3304.0,摩托车配附件,1036933.0,摩托车安全用品,1.0,50.00,50.00
2014549,6143727,2814461,3080861,196678527374758,2024-08-05 16:47:53,TH443526,1688,24080500343,XLY240804100128,1,...,裱花嘴、裱花袋、裱花枪,15.0,日用百货,10297.0,烘焙用具,1048289.0,裱花嘴、裱花袋、裱花枪,50.0,0.55,27.50
2014550,6143728,2814461,3080861,196678527423532,2024-08-05 16:47:53,TH443526,1688,24080500343,XLY240804100128,1,...,裱花嘴、裱花袋、裱花枪,15.0,日用百货,10297.0,烘焙用具,1048289.0,裱花嘴、裱花袋、裱花枪,20.0,4.20,84.00


In [10]:
data_drop.shape # จำนวนข้อมูลหลังลบค่าว่างทิ้ง

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(880845, 21)

In [11]:
data_drop[data_drop['product_quantity'] ==0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,purchases_id,cars_id,baskets_id,transferred_date,th_code,chanal_purchase,po,so,chanal_bill,...,product_name,cat_main,cat_main_name,cat_sub1,cat_sub1_name,cat_sub2,cat_sub2_name,product_quantity,product_price,total_amount
60,4129238,2251102,2340140,196678524107617,2022-01-02 12:19:56,TH13430,1688,21123000327,XHY211229100633,1,...,仿真绿植,67.0,办公、文教,201162103.0,仿真园艺,1038172.0,仿真植物,0.0,3.95,0.0
78,4129256,2251102,2340140,196678524107635,2022-01-02 12:19:56,TH13430,1688,21123000327,XHY211229100633,1,...,仿真绿植,67.0,办公、文教,201162103.0,仿真园艺,1038172.0,仿真植物,0.0,5.50,0.0
485,4129663,2251536,2342259,196678524117562,2022-01-03 16:40:28,TH544422,1688,22010300132,XLY220103100128,1,...,小学书包,1042954.0,箱包皮具,126128002.0,儿童包,1031750.0,小学书包,0.0,112.00,0.0
566,4129744,2251555,2340876,196678524110681,2022-01-03 14:48:47,TH631771,1688,22010300151,LY211230100573,2,...,休闲裤,10166.0,女装,125748001.0,女式休闲裤,1031871.0,休闲裤,0.0,15.99,0.0
727,4129905,2251611,2341115,196678524111837,2022-01-03 14:48:49,TH041271,1688,22010300207,XLY211231100175,2,...,办公用胶带,67.0,办公、文教,2113.0,装订、胶粘、桌面用品,124746005.0,办公用胶带,0.0,2.70,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006097,6135275,2811567,3075591,196678527393949,2024-07-31 13:05:25,TH044508,1688,24073100002,XLY240728100127,1,...,狗狗服装,122916001.0,宠物及园艺,121828001.0,狗狗及用品,121778003.0,狗狗服装,0.0,12.50,0.0
2006102,6135280,2811567,3075591,196678527393970,2024-07-31 13:05:25,TH044508,1688,24073100002,XLY240728100127,1,...,狗狗服装,122916001.0,宠物及园艺,121828001.0,狗狗及用品,121778003.0,狗狗服装,0.0,12.50,0.0
2006103,6135281,2811567,3075591,196678527393972,2024-07-31 13:05:25,TH044508,1688,24073100002,XLY240728100127,1,...,狗狗服装,122916001.0,宠物及园艺,121828001.0,狗狗及用品,121778003.0,狗狗服装,0.0,12.50,0.0
2006104,6135282,2811567,3075591,196678527393974,2024-07-31 13:05:25,TH044508,1688,24073100002,XLY240728100127,1,...,狗狗服装,122916001.0,宠物及园艺,121828001.0,狗狗及用品,121778003.0,狗狗服装,0.0,12.50,0.0


In [12]:
data_drop[data_drop['total_amount'] ==0]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,purchases_id,cars_id,baskets_id,transferred_date,th_code,chanal_purchase,po,so,chanal_bill,...,product_name,cat_main,cat_main_name,cat_sub1,cat_sub1_name,cat_sub2,cat_sub2_name,product_quantity,product_price,total_amount
60,4129238,2251102,2340140,196678524107617,2022-01-02 12:19:56,TH13430,1688,21123000327,XHY211229100633,1,...,仿真绿植,67.0,办公、文教,201162103.0,仿真园艺,1038172.0,仿真植物,0.0,3.95,0.0
78,4129256,2251102,2340140,196678524107635,2022-01-02 12:19:56,TH13430,1688,21123000327,XHY211229100633,1,...,仿真绿植,67.0,办公、文教,201162103.0,仿真园艺,1038172.0,仿真植物,0.0,5.50,0.0
485,4129663,2251536,2342259,196678524117562,2022-01-03 16:40:28,TH544422,1688,22010300132,XLY220103100128,1,...,小学书包,1042954.0,箱包皮具,126128002.0,儿童包,1031750.0,小学书包,0.0,112.00,0.0
566,4129744,2251555,2340876,196678524110681,2022-01-03 14:48:47,TH631771,1688,22010300151,LY211230100573,2,...,休闲裤,10166.0,女装,125748001.0,女式休闲裤,1031871.0,休闲裤,0.0,15.99,0.0
727,4129905,2251611,2341115,196678524111837,2022-01-03 14:48:49,TH041271,1688,22010300207,XLY211231100175,2,...,办公用胶带,67.0,办公、文教,2113.0,装订、胶粘、桌面用品,124746005.0,办公用胶带,0.0,2.70,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006097,6135275,2811567,3075591,196678527393949,2024-07-31 13:05:25,TH044508,1688,24073100002,XLY240728100127,1,...,狗狗服装,122916001.0,宠物及园艺,121828001.0,狗狗及用品,121778003.0,狗狗服装,0.0,12.50,0.0
2006102,6135280,2811567,3075591,196678527393970,2024-07-31 13:05:25,TH044508,1688,24073100002,XLY240728100127,1,...,狗狗服装,122916001.0,宠物及园艺,121828001.0,狗狗及用品,121778003.0,狗狗服装,0.0,12.50,0.0
2006103,6135281,2811567,3075591,196678527393972,2024-07-31 13:05:25,TH044508,1688,24073100002,XLY240728100127,1,...,狗狗服装,122916001.0,宠物及园艺,121828001.0,狗狗及用品,121778003.0,狗狗服装,0.0,12.50,0.0
2006104,6135282,2811567,3075591,196678527393974,2024-07-31 13:05:25,TH044508,1688,24073100002,XLY240728100127,1,...,狗狗服装,122916001.0,宠物及园艺,121828001.0,狗狗及用品,121778003.0,狗狗服装,0.0,12.50,0.0


In [13]:
# เอา product_quantity & total_amount ที่มีค่าเป็น 0 ออก
dataset = data_drop[(data_drop.product_quantity > 0) & (data_drop.total_amount >0)]
dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,purchases_id,cars_id,baskets_id,transferred_date,th_code,chanal_purchase,po,so,chanal_bill,...,product_name,cat_main,cat_main_name,cat_sub1,cat_sub1_name,cat_sub2,cat_sub2_name,product_quantity,product_price,total_amount
0,4129178,2248805,2337512,196678524095069,2022-01-02 12:21:17,TH403375,1688,21122700096,XLY211226100380,1,...,大码连衣裙,10166.0,女装,127372010.0,大码女装,1046799.0,大码连衣裙,1.0,43.06,43.06
1,4129179,2249918,2338921,196678524101901,2022-01-02 12:21:18,TH174119,1688,21122800487,XHY211228100359,1,...,线卡、线扣,57.0,电子元器件,1033273.0,布线产品,1033954.0,线卡、线扣,200.0,0.55,110.00
6,4129184,2250644,2340008,196678524106956,2022-01-02 12:19:20,TH100168,1688,21122900508,XLY211229100501,1,...,餐椅,13.0,家装、建材,123610020.0,餐厅家具,121486003.0,餐椅,12.0,53.44,641.28
7,4129185,2250661,2339780,196678524105638,2022-01-02 12:19:21,TH059370,1688,21122900525,XLY211229100273,1,...,冰袋、冰包、野餐包,15.0,日用百货,10287.0,餐具,1031741.0,冰袋、冰包、野餐包,2.0,50.00,100.00
8,4129186,2250661,2339780,196678524105639,2022-01-02 12:19:21,TH059370,1688,21122900525,XLY211229100273,1,...,冰袋、冰包、野餐包,15.0,日用百货,10287.0,餐具,1031741.0,冰袋、冰包、野餐包,1.0,50.00,50.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013645,6142823,2814180,3079908,196678527457960,2024-08-06 16:51:22,TH519272,1688,24080500062,XLY240802100720,1,...,收纳整理架、置物架,15.0,日用百货,122390004.0,整理用具,122312007.0,收纳整理架、置物架,3.0,8.00,24.00
2014392,6143570,2814392,3080565,196678527470152,2024-08-06 15:26:20,TH120467,1688,24080500274,XLY240803100494,1,...,摩托车安全用品,71.0,汽摩及配件,3304.0,摩托车配附件,1036933.0,摩托车安全用品,4.0,50.00,200.00
2014393,6143571,2814392,3080565,196678527470153,2024-08-06 15:26:20,TH120467,1688,24080500274,XLY240803100494,1,...,摩托车安全用品,71.0,汽摩及配件,3304.0,摩托车配附件,1036933.0,摩托车安全用品,1.0,50.00,50.00
2014549,6143727,2814461,3080861,196678527374758,2024-08-05 16:47:53,TH443526,1688,24080500343,XLY240804100128,1,...,裱花嘴、裱花袋、裱花枪,15.0,日用百货,10297.0,烘焙用具,1048289.0,裱花嘴、裱花袋、裱花枪,50.0,0.55,27.50


**คอลัมน์ที่นำมาใช้งาน**

  *purchase_id == รหัสคำสั่งซื้อ*

  *product_name == ชื่่อสินค้า*

  *th_code == รหัสลูกค้า*

In [14]:
# ใน 1 การสั่งซื้อลูกค้าซื้อสินค้าไปจำนวนกี่รายการ
purchase = dataset.groupby('purchases_id').count().sort_values(by=['cat_main_name'],ascending=False)
purchase.iloc[:,12:13].reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,purchases_id,cat_main_name
0,2691827,144
1,2691925,135
2,2741069,88
3,2691969,84
4,2691842,81
...,...,...
174787,2800597,1
174788,2800710,1
174789,2803543,1
174790,2803617,1


In [15]:
# ใน 1 การสั่งซื้อลูกค้าซื้อสินค้าไปจำนวนกี่รายการ โดยที่สินค้าไม่ซ้ำ
product_purchase = dataset.groupby('purchases_id')['cat_main_name'].nunique().sort_values(ascending=False)
product_purchase.reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,purchases_id,cat_main_name
0,2426847,8
1,2438748,7
2,2283806,7
3,2296661,6
4,2341773,6
...,...,...
174787,2429262,1
174788,2429265,1
174789,2429269,1
174790,2429281,1


In [16]:
# ลูกค้า 1 คนสั่งซื้อสินค้าไปกี่รายงาน
th_code = dataset.groupby('th_code').count().sort_values(by=['cat_main_name'],ascending=False)
th_code.iloc[:,12:13].reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,th_code,cat_main_name
0,TH509912,15403
1,TH124928,9692
2,TH051572,8858
3,TH306141,8583
4,TH111652,8469
...,...,...
10980,TH967440,1
10981,TH968152,1
10982,TH968398,1
10983,TH969349,1


In [17]:
#  ลูกค้า 1 คนสั่งซื้อสินค้าไปกี่รายงาน โดยที่สินค้าไม่ซ้ำ
th_code_purchase = dataset.groupby('th_code')['cat_main_name'].nunique().sort_values(ascending=False)
th_code_purchase.reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,th_code,cat_main_name
0,TH00666,33
1,TH30323,31
2,TH652464,31
3,TH413111,30
4,TH069347,30
...,...,...
10980,TH636197,1
10981,TH636129,1
10982,TH635997,1
10983,TH635960,1


In [18]:
# ลูกค้า 1 คนสั่งซื้อสินค้าไปกี่รายงาน
products = dataset.groupby('cat_main_name').count().sort_values(by=['purchases_id'],ascending=False)
products = products.iloc[:,1:2].reset_index()
products

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,cat_main_name,purchases_id
0,服饰配件、饰品,158859
1,内衣,64683
2,日用百货,63012
3,办公、文教,59325
4,运动服饰,57061
5,数码、电脑,52754
6,运动装备,44047
7,鞋,41073
8,玩具,40077
9,宠物及园艺,32079


In [19]:
transaction = dataset.groupby('purchases_id')['cat_main_name'].apply(lambda x: list(set(x))).reset_index()
transaction

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,purchases_id,cat_main_name
0,2247714,[日用百货]
1,2248805,[女装]
2,2248972,[日用百货]
3,2249424,[家居饰品]
4,2249546,[机械及行业设备]
...,...,...
174787,2812823,[内衣]
174788,2812917,[玩具]
174789,2814180,[日用百货]
174790,2814392,[汽摩及配件]


In [20]:
#set_list = dataset.groupby(['purchases_id', 'product_name'])['product_name'].count().unstack().reset_index().fillna(0).set_index('purchases_id')
#set_list

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
transaction_list = transaction['cat_main_name'].tolist()
te = TransactionEncoder()
te_ary = te.fit(transaction_list).transform(transaction_list)
transaction_set = pd.DataFrame(te_ary, columns=te.columns_)
transaction_set

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,个护/家清,五金、工具,交通运输,仪器仪表,传媒、广电,内衣,农业,冶金矿产,办公、文教,加工,...,纺织、皮革,美容护肤/彩妆,能源,运动服饰,运动装备,通信产品,钢铁,鞋,食品酒水,餐饮生鲜
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174787,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
174788,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
174789,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
174790,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
transaction_set.isnull().any()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
个护/家清,False
五金、工具,False
交通运输,False
仪器仪表,False
传媒、广电,False
内衣,False
农业,False
冶金矿产,False
办公、文教,False
加工,False


In [31]:
filter_test = dataset[dataset['cat_main_name'] == '箱包皮具']
filter_test.iloc[:,5:6].reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,th_code
0,319,TH557168
1,320,TH557168
2,321,TH557168
3,322,TH557168
4,484,TH544422
...,...,...
12456,1867686,TH066322
12457,1867687,TH066322
12458,1867688,TH066322
12459,1867689,TH066322


In [32]:
test = filter_test.groupby('cat_main_name')['th_code'].apply(lambda x: ','.join(set(x))).reset_index() \
    .assign(th_code=lambda df: df['th_code'].str.split(',')) \
    .explode('th_code').reset_index(drop=True)
test

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,cat_main_name,th_code
0,箱包皮具,TH731029
1,箱包皮具,TH10464
2,箱包皮具,TH429550
3,箱包皮具,TH25830
4,箱包皮具,TH116333
...,...,...
440,箱包皮具,TH017693
441,箱包皮具,TH786369
442,箱包皮具,TH173393
443,箱包皮具,TH055415


In [33]:
frequent_itemsets = apriori(transaction_set,min_support = 0.005, use_colnames=True) ## 0.001% ของรายการสินค้าทั้งหมดที่มีคนซื้อมากกว่า 1 %
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.015573,(个护/家清)
1,0.018679,(五金、工具)
2,0.006379,(仪器仪表)
3,0.048034,(内衣)
4,0.064746,(办公、文教)
5,0.049865,(包装)
6,0.019864,(女装)
7,0.008845,(安全、防护)
8,0.034161,(宠物及园艺)
9,0.018102,(家居饰品)


In [34]:
#for result in frequent_itemsets:
  #print(result)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.005)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


**-----------------------------------------------------------------------------------------------------------------------**